In [1]:
%%cmd
pip install cmake
pip install "C:\Users\owner\Downloads\Dlib-python whl packages\Dlib-python whl packages\dlib-19.22.99-cp39-cp39-win_amd64.whl"
pip install face-recognition
pip install opencv-python

Microsoft Windows [Version 10.0.18363.418]
(c) 2019 Microsoft Corporation. All rights reserved.

(base) C:\Users\owner>pip install cmake

(base) C:\Users\owner>pip install "C:\Users\owner\Downloads\Dlib-python whl packages\Dlib-python whl packages\dlib-19.22.99-cp39-cp39-win_amd64.whl"
Processing c:\users\owner\downloads\dlib-python whl packages\dlib-python whl packages\dlib-19.22.99-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

(base) C:\Users\owner>pip install face-recognition

(base) C:\Users\owner>pip install opencv-python

(base) C:\Users\owner>

In [2]:
import cmake
import dlib
import face_recognition as face_rec
import os
import cv2
import numpy as np
import pandas as pd
import sqlite3
from openpyxl import load_workbook
from datetime import date

In [3]:
path= r"C:\Users\owner\anaconda3\ATTENDANCE SYSTEM PROJECT\train_data_set"
data_base= os.listdir(path)
image=[]
persons_name=[]
for img_id in data_base:
    img_on=cv2.imread(f"{path}/{img_id}")
    image.append(img_on)
    persons_name.append(os.path.splitext(img_id)[0])

In [4]:
today= date.today()
todays_date=today.strftime("%d/%m/%Y")
day=int(today.strftime("%d"))
xlsx_ref=["C","D","E","F","G",'H',"I","J","K","L","M","N","O","P","Q","R",
          "S","T","U","V","W","X","Y","Z","AA","AB","AC","AD","AE","AF","AG"]


In [5]:
def encode_image(image):
    encode_list=[]
    for img in image:
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB )
        encode= face_rec.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def update_register(path):
    exl= pd.read_excel(path)
    sql = sqlite3.connect(r"C:\Users\owner\Desktop\Training_Database.db")
    sql_cur = sql.cursor()
    sql_cur.execute('CREATE TABLE IF NOT EXISTS School_Attendance (SERIAL_NO INTEGER, STUDENTS_NAME TEXT)')
    sql.commit()
    exl.to_sql("School_Attendance", con=sql, if_exists='replace')
    print("REGISTER UPDATED")
    
def find_name(name):
    register=sqlite3.connect(r"C:\Users\owner\Desktop\Training_Database.db")
    cur= register.cursor()
    serial=cur.execute("SELECT SERIAL_NO from School_Attendance where STUDENTS_NAME =(?)", (name,))
    serial=serial.fetchone()
    serial=str(serial[0]+1)
    return serial


In [6]:
img_in_database = encode_image(image)
path=r"C:\Users\owner\anaconda3\ATTENDANCE SYSTEM PROJECT\School_Attendance.xlsx"
#note the path need to be updated if paraventure you change the location of the School_Attendance.
update_register(path)

REGISTER UPDATED


In [8]:
cap=cv2.VideoCapture(0)
while True:
    ind,frame= cap.read()
    #frame=cv2.resize(frame, (0,0), None, 0.5, 0.5)
    if len(frame.shape) == 2:
        frame = cv2.cvtColor(frame,cv2.COLOR_GRAY2BGR)
    face_on_cam= face_rec.face_locations(frame)
    encode_face= face_rec.face_encodings(frame, face_on_cam)
    
    for face_encode, face_location in zip(encode_face, face_on_cam):
        recognice= face_rec.compare_faces(img_in_database, face_encode)
        doubt= face_rec.face_distance(img_in_database, face_encode)
        y1, x2, y2, x1 =face_location
        a=int(x1+(x2-x1)/2)
        b=int(y1+(y2-y1)/2)
        c=int((x2-x1)/2)+25
        choice= np.argmin(doubt)
        if recognice[choice]:
            name=persons_name[choice].upper()
            #cv2.rectangle(frame, (x1,y1),(x2,y2), (0,0,0),2)
            serial=find_name(name)
            workbook= load_workbook(path)
            sheet=workbook.active
            sheet[xlsx_ref[day-1]+'1']= todays_date
            sheet[xlsx_ref[day-1]+(serial)]= "present"
            workbook.save(path)
            workbook.close()
            #cv2.rectangle(frame, (x1,y1),(x2,y2), (0,0,0),2)
            cv2.circle(frame, (a,b ),c,(255,255,255),3)
            cv2.putText(frame, name, (x1,y1-30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,255,255),1)
    cv2.imshow('webcam',frame)
    cv2.waitKey(300)

TypeError: 'NoneType' object is not subscriptable